In [4]:
# this script calculates the imbalance according to the measures: imbalance ratio, imbalance degree, imbalance score, mean imbalance ratio.
# Additionally, for experiment 1 the initial data seen below is iteratively adjusted and used as input for the four metrics
data = {
    'Edema': [5000, 15000],
    'Cardiomegaly': [5000, 15000],
    'Atelectasis': [5000, 15000]
}
# Furthermore, the script generates random group splits for studies 2-4 and calculates the IS for this 

In [18]:
# import and utility functions
import imbalance_degree.imbalance_degree as ib
import pandas as pd
import numpy as np
from IS import mmult
def aggregate(df,labels, byGender):
     #Initialize an empty dictionary to store the aggregation functions
    aggregation_functions = {}

     #Iterate over the columns and add them to the aggregation functions dictionary
    for column in labels:
        aggregation_functions[column] = 'sum'
    if byGender:
         #Perform the dynamic aggregation
        result = df.groupby(['gender']).agg(aggregation_functions)
    else:
        result = pd.DataFrame(df.agg(aggregation_functions)).T

    return result

# IS check

In [19]:
data = {
    'Edema': [15000, 15000],
    'Cardiomegaly': [45000, 15000],
    'Atelectasis': [15000, 15000]
}

index = ['0', '1']

df = pd.DataFrame(data, index=index)

In [20]:
result = mmult(df)
result

c:\Users\rankl\Documents\uni\Thesis\Development\analysis_modelDesign_dataDist\experiments\ImbalanceScore\IS.py:76: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  matrix_mul = np.matmul(df_IR_gender,df_IR_disease)


0    4.0
dtype: float64

# Experiment 1

## IS

In [ ]:
data = {
    'Edema': [5000, 15000],
    'Cardiomegaly': [5000, 15000],
    'Atelectasis': [5000, 15000]
}

index = ['0', '1']

df = pd.DataFrame(data, index=index)


result_df = pd.DataFrame(columns=['mmult_Result'])
result = mmult(df)


result_df = result_df.append({'mmult_Result': result.values}, ignore_index=True)

# Loop through each class
for col in df.columns:
    current_freq = df.at['0', col]
    while current_freq <= 15000:
        current_freq += 5000
        if current_freq > 15000:
            continue
        else:
            df.at['0', col] = current_freq
        result = mmult(df)
        result_df = result_df.append({'mmult_Result': result.values}, ignore_index=True)
        
#result_df.to_excel('result_interpretation.xlsx', index=False)

## Imbalance degree

In [15]:
data = {
    'Edema': [5000, 15000],
    'Cardiomegaly': [5000, 15000],
    'Atelectasis': [5000, 15000]
}

index = ['0', '1']

df = pd.DataFrame(data, index=index)
aggregated_df = pd.DataFrame(columns=df.columns)
aggregated_result = df.sum()
aggregated_df = aggregated_df.append(aggregated_result, ignore_index=True)

for col in df.columns:
    current_freq = df.at['0', col]
    while current_freq <= 15000:
        current_freq += 5000
        if current_freq > 15000:
            continue
        else:
            df.at['0', col] = current_freq
        
        aggregated_result = df.sum()
        aggregated_df = aggregated_df.append(aggregated_result, ignore_index=True)

            

# Print the aggregated results
print(aggregated_df)

# List to store data for each class
#class_data = {col: [] for col in df.columns}
class_data = [] #['classes']
results = []
# Iterate through each row in the aggregated DataFrame
for index, row in aggregated_df.iterrows():
    class_names = row.index
    class_frequencies = row.values

    # Iterate through each class and its frequency
    for class_outer, frequency in zip(class_names, class_frequencies):
        class_data.extend([class_names.get_loc(class_outer)] * int(frequency))

    results.append(ib.imbalance_degree(class_data, distance="EU"))
    
    class_data = []
result_df = pd.DataFrame(data=results,columns=["ID"])
#result_df.to_excel('result_ID.xlsx', index=False)

   Edema Cardiomegaly Atelectasis
0  20000        20000       20000
1  25000        20000       20000
2  30000        20000       20000
3  30000        25000       20000
4  30000        30000       20000
5  30000        30000       25000
6  30000        30000       30000


C:\Users\rankl\AppData\Local\Temp\ipykernel_19340\89403381.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aggregated_df = aggregated_df.append(aggregated_result, ignore_index=True)
C:\Users\rankl\AppData\Local\Temp\ipykernel_19340\89403381.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aggregated_df = aggregated_df.append(aggregated_result, ignore_index=True)
C:\Users\rankl\AppData\Local\Temp\ipykernel_19340\89403381.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aggregated_df = aggregated_df.append(aggregated_result, ignore_index=True)
C:\Users\rankl\AppData\Local\Temp\ipykernel_19340\89403381.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. 

## Mean Imbalance ratio, Imbalance per label

In [16]:
data = {
    'Edema': [5000, 15000],
    'Cardiomegaly': [5000, 15000],
    'Atelectasis': [5000, 15000]
}

index = ['0', '1']

df = pd.DataFrame(data, index=index)

aggregated_df = pd.DataFrame(columns=df.columns)
aggregated_result = df.sum()
aggregated_df = aggregated_df.append(aggregated_result, ignore_index=True)


for col in df.columns:
    current_freq = df.at['0', col]
    while current_freq <= 15000:
        current_freq += 5000
        if current_freq > 15000:
            continue
        else:
            df.at['0', col] = current_freq
        aggregated_result = df.sum()
        aggregated_df = aggregated_df.append(aggregated_result, ignore_index=True)

IRperLabel_values = []
mean_IRpLabel = []
for index, row in aggregated_df.iterrows():
    output = max(row.values)/row.values
    IRperLabel_values.append(output)
    mean_IRpLabel.append(np.mean(output))
        
result_df = pd.DataFrame(data=zip(IRperLabel_values,mean_IRpLabel))
result_df


C:\Users\rankl\AppData\Local\Temp\ipykernel_19340\3322541807.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aggregated_df = aggregated_df.append(aggregated_result, ignore_index=True)
C:\Users\rankl\AppData\Local\Temp\ipykernel_19340\3322541807.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aggregated_df = aggregated_df.append(aggregated_result, ignore_index=True)
C:\Users\rankl\AppData\Local\Temp\ipykernel_19340\3322541807.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aggregated_df = aggregated_df.append(aggregated_result, ignore_index=True)
C:\Users\rankl\AppData\Local\Temp\ipykernel_19340\3322541807.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

,0,1
0,"[1.0, 1.0, 1.0]",1.000000
1,"[1.0, 1.25, 1.25]",1.166667
2,"[1.0, 1.5, 1.5]",1.333333
3,"[1.0, 1.2, 1.5]",1.233333
4,"[1.0, 1.0, 1.5]",1.166667
5,"[1.0, 1.0, 1.2]",1.066667
6,"[1.0, 1.0, 1.0]",1.000000


# Experiments 2 - 4: Random initialisation across three groups as input for IS

In [5]:
# this initial data is used as input for IR, mIR, ID
data = {
    'Edema': [5000,10000,15000, 15000, 25000, 30000, 35000],
    'Cardiomegaly': [2500,5000,7500, 10000, 12500, 15000, 17500],
    'Atelectasis': [7500,12500,17500, 22500, 27500, 32500, 37500]
}

index = list(range(7))

df_class_distributions = pd.DataFrame(data, index=index)

# Print the transformed DataFrame
df_class_distributions.to_clipboard()

In [ ]:
excel_index = 1
with pd.ExcelWriter('output_results_42.xlsx') as writer:

    for index, row in df_class_distributions.iterrows():
        # specify seed: in the experiments we use 2, 23, 42
        np.random.seed(42)
        new_df = pd.DataFrame([row], columns=df_class_distributions.columns, index=[0])
        class_frequencies_np = np.array(row.values)


        total_values = {col: value for col, value in new_df.iloc[0].items()}

        num_groups = 3
        groups = []
        mean_value = class_frequencies_np / 3

        remaining_values = class_frequencies_np.copy()
        
        group_values = {group_index: np.zeros_like(new_df.values[0]) for group_index in range(num_groups)}
    
        for group_index in range(num_groups - 1):
            range_percentage = 0.45  # 30% around the mean
            

            lower_bound = mean_value * (1 - range_percentage)
            upper_bound = mean_value * (1 + range_percentage)

            group_row = np.random.randint(lower_bound, upper_bound + 1, size=num_groups)


            values =  np.minimum(group_row, remaining_values)
            group_values[group_index] = values
            
            remaining_values -= group_row
            mean_value = remaining_values / (num_groups - (group_index + 1))

        group_values[2] = remaining_values

        # output to excel
        # Create a new DataFrame with the split data and groups
        result_df = pd.DataFrame(group_values, index=['Edema','Cardiomegaly','Atelectasis'])
        result_df = result_df.T
        result_df.to_excel(writer, sheet_name='Group_Sheet', startrow=excel_index, index=False)

        
        imbalance_score = mmult(result_df)
        imbalance_score = imbalance_score.T
        imbalance_score.to_excel(writer, sheet_name='Group_Sheet',startrow=excel_index, startcol=5, index=False)
        excel_index +=20
